# Cvičení 5

Tématem tohoto cvičení je výpočer QR rozkladu matice A pomocí několika různých metod. 

## QR rozklad - pomocí Gram-Schmidtova ortogonalizačního procesu

QR rozklad převádí matici $\mathsf{A}$ na součin ortogonální a horní trojúhelníkové matice,
$$\mathsf{A}=\mathsf{Q}\mathsf{R},$$
kde $Q$ je ortogonální matice a $R$ horní trojúhelníková matice. Můžeme jej popsat např. následujícím pseudokódem:

```
n = size(A)
R(1,1) = norm (A(:,1))
Q(:,1) = A(:,1) / R(1,1)

for k = 2, ..., n do
    z = A(:, k)
    [R(1, k), ..., R(k-1, k)] = Q(:, k-1)^T*z
    z = z - Q(:, k-1)*[R(1, k), ..., R(k-1, k)]

    R[k, k] = norm(z)
    Q[:, k] = z / R[k, k]
end for
```

In [15]:
import numpy as np

def qr_decomposition(A):
    m, n = A.shape
    Q = np.zeros((m, n)) 
    R = np.zeros((n, n)) 
    
    # UKOL: rozmyslete si, proc by bylo vhodnejsi pouzit np.zeros_like(A) misto np.zeros((m, n))
    # Q = np.zeros_like(A, dtype=float)
    # R = np.zeros((n, n), dtype=float)
    
    for k in range(n):
        z = A[:, k]
        
        for j in range(k): # Ortogonalizace
            R[j, k] = np.dot(Q[:, j].T, z)
            z = z - R[j, k] * Q[:, j]

        print(f"{k+1}. Ortogonální sloupec: {z}")

        R[k, k] = np.linalg.norm(z)
        if R[k, k] != 0:
            Q[:, k] = z / R[k, k]
        else:
            Q[:, k] = np.zeros_like(z)  # Uložení nulového vektoru do příslušného sloupce Q

        print(f"{k+1}. Ortonormalizovaný sloupec: {Q[:, k]}")
        print(f"{k+1}. Sloupec R: {R[:, k]}")
    
    return Q, R

A = np.array([[3, 1],
              [6, 2],
              [0, 2]])

Q, R = qr_decomposition(A)
print(Q)
print(R)
print(Q@R - A)

1. Ortogonální sloupec: [3 6 0]
1. Ortonormalizovaný sloupec: [0.4472136  0.89442719 0.        ]
1. Sloupec R: [6.70820393 0.        ]
2. Ortogonální sloupec: [0. 0. 2.]
2. Ortonormalizovaný sloupec: [0. 0. 1.]
2. Sloupec R: [2.23606798 2.        ]
[[0.4472136  0.        ]
 [0.89442719 0.        ]
 [0.         1.        ]]
[[6.70820393 2.23606798]
 [0.         2.        ]]
[[0. 0.]
 [0. 0.]
 [0. 0.]]


ÚKOL: Pomocí vhodných výpisů ověřte správnou funkčnost kódu ve srovnání s pseudokódem, tzn. že není potřeba prvky R[0,0] a Q[:,0] počítat zvlášť mimo cyklus.

```
    Pro k = 0:
        z = A[:, 0] – první sloupec matice A je vzat přímo.
        Smyčka for j in range(k) se přeskočí, protože range(0) je prázdná.
        R[0, 0] = np.linalg.norm(z) vypočítá normu prvního sloupce.
        Q[:, 0] = z / R[0, 0] normalizuje první sloupec a uloží ho do Q.

    Pro další k > 0:
        Vypočítají se projekce na předchozí vektory a odečtou se od aktuálního sloupce.
        Poté se výsledek normalizuje a uloží se do aktuálního sloupce Q.
```

In [16]:
# Testovací příklad
# A = np.random.rand(5, 5) # Náhodná matice 5x5
A = np.array([[1.0, 2.0, 3.0],
             [4.0, 5.0, 6.0],
             [7.0, 8.0, 9.0]])
print("Matice A:\n", A)
Q, R = qr_decomposition(A)
print("Q:", Q)
print("R:", R)

# Ověření správnosti našeho QR rozkladu
A_reconstructed = Q @ R
print("Rekonstruovaná A:\n", A_reconstructed)

# A se musi rovnat QR
print("Rekonstruovaná A - A:\n", A_reconstructed-A)

# Porovnání s původní maticí A
if np.allclose(A, A_reconstructed):
    print("QR rozklad je správný!")
else:
    print("QR rozklad není správný.")

Matice A:
 [[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]]
1. Ortogonální sloupec: [1. 4. 7.]
1. Ortonormalizovaný sloupec: [0.12309149 0.49236596 0.86164044]
1. Sloupec R: [8.1240384 0.        0.       ]
2. Ortogonální sloupec: [ 0.81818182  0.27272727 -0.27272727]
2. Ortonormalizovaný sloupec: [ 0.90453403  0.30151134 -0.30151134]
2. Sloupec R: [9.6011363  0.90453403 0.        ]
3. Ortogonální sloupec: [0. 0. 0.]
3. Ortonormalizovaný sloupec: [0. 0. 0.]
3. Sloupec R: [11.07823419  1.80906807  0.        ]
Q: [[ 0.12309149  0.90453403  0.        ]
 [ 0.49236596  0.30151134  0.        ]
 [ 0.86164044 -0.30151134  0.        ]]
R: [[ 8.1240384   9.6011363  11.07823419]
 [ 0.          0.90453403  1.80906807]
 [ 0.          0.          0.        ]]
Rekonstruovaná A:
 [[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]]
Rekonstruovaná A - A:
 [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
QR rozklad je správný!


## QR rozklad - Givensova transformace

Mějme trojúhelník s odvěsnami $a$, $b$ a přeponou $r$, přičemž $\|r\| = \sqrt{a^2 + b^2}$. Kosinus úhlu $\alpha$ mezi stranami $a$ a $r$ spočítáme jako:

$\cos(\alpha) = \frac{a}{\sqrt{a^2+b^2}}$
a sinus úhlu $\alpha$ jako 
$\sin(\alpha) = \frac{b}{\sqrt{a^2+b^2}}$. 

Matice Givensovy rotace pak vypadá takto: 

$$
G(\alpha) = \begin{pmatrix}
\cos(\alpha) & -\sin(\alpha) \\
\sin(\alpha) & \cos(\alpha)
\end{pmatrix} 
$$


In [48]:
# UKOL: doplnte funkci givens_rotation
# Funkce by mela vracet dvojici c (kosinus), s (sinus)

def givens_rotation(a, b):
    if b == 0:
        c, s = 1, 0
    else:
        r = np.sqrt(a**2 + b**2)
        c = a / r
        s = b / r
    return c, s

In [18]:
def qr_decomposition_givens(A):
    m, n = A.shape
    Q = np.eye(m)
    R = A.copy()
    
    for j in range(n):
        for i in range(m-1, j, -1):
            c, s = givens_rotation(R[i-1, j], R[i, j])
            
            G = np.eye(m)
            G[[i-1, i], [i-1, i]] = c
            G[i, i-1], G[i-1, i] = s, -s
            
            R = G @ R
            Q = Q @ G.T
    
    return Q, R

In [49]:
# Testovací příklad
A = np.random.rand(5, 5) # Náhodná matice 5x5
print("Matice A:\n", A)
Q, R = qr_decomposition_givens(A)
# print("Q:", Q)
# print("R:", R)

# Ověření správnosti našeho QR rozkladu
A_reconstructed = Q @ R
print("Rekonstruovaná A:\n", A_reconstructed)

# A se musi rovnat QR
print("Rekonstruovaná A - A:\n", A_reconstructed-A)

# Porovnání s původní maticí A
if np.allclose(A, A_reconstructed):
    print("QR rozklad pomocí Givensových rotací je správný!")
else:
    print("QR rozklad není správný.")

Matice A:
 [[0.74738943 0.30375545 0.1780991  0.7614157  0.77814131]
 [0.78178097 0.26718112 0.61258187 0.12484332 0.73872787]
 [0.1425306  0.2631871  0.03799895 0.36821017 0.30844563]
 [0.33967146 0.19552003 0.63762746 0.57049749 0.76261596]
 [0.4144717  0.30958022 0.53092009 0.17792164 0.85723938]]
Rekonstruovaná A:
 [[0.74738943 0.30375545 0.1780991  0.7614157  0.77814131]
 [0.78178097 0.26718112 0.61258187 0.12484332 0.73872787]
 [0.1425306  0.2631871  0.03799895 0.36821017 0.30844563]
 [0.33967146 0.19552003 0.63762746 0.57049749 0.76261596]
 [0.4144717  0.30958022 0.53092009 0.17792164 0.85723938]]
Rekonstruovaná A - A:
 [[-2.22044605e-16 -1.11022302e-16 -2.77555756e-17 -2.22044605e-16
  -2.22044605e-16]
 [ 0.00000000e+00 -1.11022302e-16  1.11022302e-16 -1.11022302e-16
  -1.11022302e-16]
 [-2.77555756e-17 -1.11022302e-16 -6.24500451e-17 -1.11022302e-16
  -1.11022302e-16]
 [-1.66533454e-16 -5.55111512e-17 -3.33066907e-16 -2.22044605e-16
  -2.22044605e-16]
 [-2.22044605e-16 -1.1102

## QR rozklad - Householderova reflexe (Householderovy transformace)

Dalším ze způsobů, jak vypočítat QR rozklad, je Householderova reflexe. V následující části cvičení se zaměříme na využití Householderovy reflexe při nulování prvků nějakého daného vektoru.

Připomeňme, že Householderovou maticí rozumíme matici $\mathsf{P}$, která daný vektor $\mathbf{x}$ projektuje na vektor $\mathsf{P}\mathbf{x}=(\|\mathbf{x}\|, 0, 0, \ldots, 0)^T$ získáme jako
$$
\mathsf{P} = \mathsf{I} - 2\frac{\mathbf{v}\mathbf{v}^T}{\mathbf{v}^T\mathbf{v}},
$$
kde pro vektor $\mathbf{v}$ platí $\mathbf{v} = -\mathrm{sign}(x_1)\|\mathbf{x}\|\mathbf{e}_1-\mathbf{x}$.

(Pozor, na přednášce: $\mathbf{n} = \frac{1}{2} ( \mathbf{x} - \|\mathbf{x}\|\mathbf{e}_1 ) $ !!!).

Nejprve tedy vytvoříme pomocnou funkci `householder`, která pro vstupní vektor `x` vygeneruje Householderovu matici nulující druhý až poslední prvek.

In [80]:
import numpy as np

def householder(x):
    """
    Vrátí matici Householderovy transformace, která vynuluje prvky na druhé až 
    poslední pozici vstupního vektoru.
    x: input vector
    P: matice zrcadlení (Householderova matice)
    """

    # UKOL: Doplnte kod funkce householder.
    # Potrebujete:
    # 1. urcit delku vektoru x
    n = x.shape[0]

    # 2. sestavit vektor e_1 = (1, 0, 0, ..., 0) odpovidajici delky
    e_1 = np.zeros_like(x)
    e_1[0] = 1

    # 3. sestavit vektor v
    v = -np.sign(x[0]) * np.linalg.norm(x) * e_1 - x

    # 4. pomoci nej sestavit matici P a vratit ji
    P = np.eye(n) - 2 * (v @ v.T) / (v.T @ v)

    return P

x = np.arange(1, 5)
x = x.reshape(x.shape[0], 1)
print(x)
P = householder(x)
#print(P)
print(P @ x)

[[1]
 [2]
 [3]
 [4]]
[[-5.47722558e+00]
 [ 1.11022302e-16]
 [ 2.22044605e-16]
 [ 4.44089210e-16]]


In [79]:
# UKOL: Otestujte vasi metodu. Vytvorte nahodny sloupcovy vektor, matici P
# a ujistete se, ze po aplikaci matice na vektor ziskate vektor odpovidajicich vlastnosti
# (tzn. ve tvaru (||x||, 0, 0, ..., 0) )

x = np.random.rand(5, 1)
print("Vektor x: \n", x)
P = householder(x)
Px = P@x
print("Projekce Px: \n", Px)
print("Norma vektoru x: \n", np.linalg.norm(x))
print("Norma vektoru Px: \n", np.linalg.norm(Px))

Vektor x: 
 [[0.91981104]
 [0.25664476]
 [0.83227071]
 [0.74435741]
 [0.95062645]]
Projekce Px: 
 [[-1.74995772e+00]
 [-1.75318489e-17]
 [-2.32325472e-16]
 [ 3.04759220e-18]
 [-1.00325278e-16]]
Norma vektoru x: 
 1.7499577192894509
Norma vektoru Px: 
 1.749957719289451


# Domácí úkol č. 3

Doplňte následující kód funkce `my_qr_householder`, která implementuje QR rozklad s použitím Householderovy transformace. Vyjděte z pseudokódu poskytnutého v přednáškách, ale modifikujte kód tak, aby využíval vámi vytvořenou metodu `householder` k výpočtu transformační matice.

Nezapomeňte, že před aplikací musíte transformační matici umístit na vhodnou pozici v jednotkové matici, aby modifikovala pouze odpovídající prvky daného sloupce.

In [ ]:
def my_qr_householder(A):
    """
    Funkce vrátí QR rozklad čtvercové matice A
    
    A: Vstupní matice
    vraci Tuple (Q, R), kde Q je ortogonální matice a R je horní trojúhelníková matice
    """
   
    m, n = A.shape
    if m != n:
        raise ValueError("Matice neni ctvercova!")

    Q = np.eye(m)
    R = A.copy()

    # doplnte cyklus pres sloupce
    # pro prislusnou cast aktualniho sloupce i sestavte transformacni matici P,
    # updatujte R a Q
        

    return Q, R

In [ ]:
# Otestujte vasi implementaci na malem prikladu:

A = np.random.rand(5, 5)
print("Matice A:\n", A)

Q, R = my_qr_householder(A)

# Ověření správnosti našeho QR rozkladu
A_reconstructed = Q @ R
print("Rekonstruovaná A:\n", A_reconstructed)

# A se musi rovnat QR
print("Rekonstruovaná A - A:\n", A_reconstructed-A)

# Porovnání s původní maticí A
if np.allclose(A, A_reconstructed):
    print("QR rozklad pomocí Householderovy reflexe je správný!")
else:
    print("QR rozklad není správný.")